### 자모 테스트

In [1]:
from gensim.models import FastText

# 자모 변환된 한국어 문서를 불러오기
with open("corpus.txt", "r", encoding="utf-8") as f:
    sentences = [line.strip().split() for line in f.readlines()] 

# FastText 모델 학습
model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1, epochs=10)

# 학습한 모델 저장
model.save("fasttext_jamo.model")
print("FastText 자모 단위 모델 학습 완료!")


FastText 자모 단위 모델 학습 완료!


In [ ]:
from gensim.models import FastText

# 학습된 모델 불러오기
model = FastText.load("fasttext_jamo.model")

# 테스트 단어 (자모 분리)
test_word = decompose_korean("자연어")
print(f"자모 변환된 단어: {test_word}")

# 단어 벡터 출력
print(f"'{test_word}'의 벡터 값:\n", model.wv[test_word])

### JSON TEST

In [ ]:
import json
from gensim.models import FastText
from jamo import h2j, j2hcj
import re

# 숫자 뒤에 붙은 한글을 유지하면서 숫자는 단독 토큰으로 변환
def decompose_korean(text):
    """한글은 자모 변환, 숫자는 단독 토큰으로 유지하되 숫자 뒤의 한글은 유지"""
    if isinstance(text, str):
        tokens = re.findall(r'\d+|[^\d\s]+', text)  # 숫자 + 한글 분리 (숫자 뒤 한글 포함)
        processed_tokens = []

        for i, token in enumerate(tokens):
            if token.isdigit():  # 숫자는 그대로 유지
                processed_tokens.append(token)
                if i < len(tokens) - 1 and not tokens[i + 1].isdigit():  # 숫자 뒤 한글이 있으면 추가
                    processed_tokens.append("".join(j2hcj(h2j(tokens[i + 1]))))
            elif not token.isdigit() and (i == 0 or not tokens[i - 1].isdigit()):  # 숫자가 아닌 한글만 변환
                processed_tokens.append("".join(j2hcj(h2j(token))))

        return " ".join(processed_tokens)
    return text

# JSON 파일에서 데이터 불러오기 및 변환
def load_json_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 모든 데이터를 자모 변환 (숫자는 그대로 유지)
    def recursive_decompose(obj):
        if isinstance(obj, dict):  # 딕셔너리 처리
            return [recursive_decompose(value) for value in obj.values()]
        elif isinstance(obj, list):  # 리스트 처리
            return [recursive_decompose(item) for item in obj]
        elif isinstance(obj, str):  # 문자열 변환
            return decompose_korean(obj).split()  # 공백 기준으로 단어 분리
        else:
            return []

    sentences = recursive_decompose(data)

    # 리스트 평탄화 (중첩 리스트 제거)
    flat_sentences = [item for sublist in sentences for item in sublist if isinstance(sublist, list)]
    return flat_sentences

# JSON 데이터 불러오기 (한글이 자모 변환되지 않은 JSON)
json_path = "./output_jamo.json"  # 학습할 JSON 파일 경로
sentences = load_json_data(json_path)

# FastText 모델 학습 (숫자 포함)
model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1, epochs=20)

# 학습한 모델 저장
model_path = "./fasttext_jamo_with_numbers.model"
model.save(model_path)
print(f"FastText 자모 변환 + 숫자 유지 모델 학습 완료! 저장 경로: {model_path}")


In [ ]:
# 학습 데이터 샘플 10개만 출력
print("FastText 학습 데이터 (sentences) 샘플:")
for i, sentence in enumerate(sentences[:10]):  # 앞 10개 문장만 출력
    print(f"{i+1}: {sentence}")


📌 FastText 학습 데이터 (sentences) 샘플:
1: ['ㅈㅏㄱㄱㅏㅅㅏㅈㅣㄴ']
2: ['作家寫眞']
3: ['Selfportrait']
4: ['ㅎㅏㄴㄱㅣㅅㅓㄱ']
5: ['HAN', 'Kisuk']
6: []
7: ['1960']
8: ['41×51']
9: ['ㅈㅗㅇㅇㅣㅇㅔ', 'ㅈㅔㄹㄹㅏㅌㅣㄴㅅㅣㄹㅂㅓㅍㅡㄹㅣㄴㅌㅡ']
10: ['ㅅㅏㅈㅣㄴ']


In [27]:
print(f"\n 총 {len(sentences)}개의 문장이 학습 데이터로 사용됩니다.")


 총 132348개의 문장이 학습 데이터로 사용됩니다.


In [ ]:
import numpy as np
from gensim.models import FastText
from jamo import h2j, j2hcj
from sklearn.metrics.pairwise import cosine_similarity


# 문장 벡터를 생성하는 함수 (각 단어의 평균 벡터)
def sentence_to_vector(sentence, model):
    tokens = decompose_korean(sentence)  # 자모 변환
    word_vectors = []

    for token in tokens:
        if token in model.wv:
            word_vectors.append(model.wv[token])  # FastText 벡터 가져오기

    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # 빈 문장일 경우 0 벡터 반환

    return np.mean(word_vectors, axis=0)  # 모든 단어 벡터 평균 계산

In [ ]:
# 두 문장의 코사인 유사도 계산
def calculate_similarity(sentence1, sentence2, model):
    vec1 = sentence_to_vector(sentence1, model)
    vec2 = sentence_to_vector(sentence2, model)
    return cosine_similarity([vec1], [vec2])[0][0]  # 코사인 유사도 계산

In [ ]:
# 두 문장에서 숫자 비교 및 패널티 적용 함수
def compare_numbers(sentence1, sentence2):
    """두 문장의 숫자를 비교하고 다르면 패널티 적용"""
    numbers1 = set(re.findall(r'\d+', sentence1))  # 숫자 추출
    numbers2 = set(re.findall(r'\d+', sentence2))

    if numbers1 and numbers2 and numbers1 != numbers2:  # 숫자가 존재하지만 다르면
        return 0.1  # 패널티 (유사도를 낮춤)
    return 1.0  # 그대로 유지

# 커스텀 FastText 유사도 함수 (숫자 고려)
def custom_similarity(text1, text2, model):
    """FastText 기반 유사도 계산, 숫자가 다르면 패널티 적용"""
    tokens1 = text1.split()
    tokens2 = text2.split()

    # FastText 평균 벡터 계산 (해당 단어가 벡터에 있는 경우만)
    vec1 = np.mean([model.wv[word] for word in tokens1 if word in model.wv], axis=0, keepdims=True)
    vec2 = np.mean([model.wv[word] for word in tokens2 if word in model.wv], axis=0, keepdims=True)

    if vec1.shape[0] == 0 or vec2.shape[0] == 0:
        return 0.0  # 한쪽 문장이라도 FastText에 없는 단어만 있으면 유사도 0

    # 코사인 유사도 계산
    similarity = np.dot(vec1, vec2.T) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    
    # 숫자 비교 후 패널티 적용
    penalty = compare_numbers(text1, text2)
    final_score = similarity * penalty

    return final_score


In [ ]:
# 학습된 FastText 모델 로드
model = FastText.load("./fasttext_jamo_with_numbers.model")

# RAG 정답과 LLM 응답 예제
rag_answer = """
이중섭의 토끼풀 작품은 1941년에 만들어졌어요.
"""

llm_response = """
이중섭의 토끼풀 작품의 1950년에 만들어졌어요.
"""


# 유사도 계산
similarity = calculate_similarity(rag_answer, llm_response, model)

print(f"🔹 RAG 정답과 LLM 응답의 유사도: {similarity:.4f}")


🔹 RAG 정답과 LLM 응답의 유사도: 0.9837


In [ ]:
# 점수범위 -1 ~ 1 
# 1: 완전 같음
# 0: 완전 무관
# -1: 반대되는 의미이지만 거의 발생 x

# 0.7681, 0.2348, 0.8179, 0.8207, 0.7460